In [2]:
DATASET_PATH = "/home/jincan/long_seg/3DUnet/Data/Imaging_for_3DUNet_single_new/train"

import glob
import numpy as np

t1ce_list = sorted(glob.glob(f"{DATASET_PATH}/*/*image.nii.gz"))
mask_list = sorted(glob.glob(f"{DATASET_PATH}/*/*mask.nii.gz"))

print("t1ce list: ", len(t1ce_list))
print("Mask list: ", len(mask_list))

t1ce list:  99
Mask list:  99


In [3]:
import os
from tqdm import tqdm
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn.functional as F


base_dir = "/home/jincan/long_seg/my3DUNet/Data/Brats2024/single_t1c_noCrop"

scaler = MinMaxScaler()
for idx in tqdm(
    range(len(t1ce_list)), desc="Preparing to stack, crop and save", unit="file"
):
    temp_image_t1ce = nib.load(t1ce_list[idx]).get_fdata()
    shape = temp_image_t1ce.shape
    temp_image_t1ce = scaler.fit_transform(
        temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])
    )
    temp_image_t1ce = temp_image_t1ce.reshape(shape)
 
    # temp_image_t2 = nib.load(t2_list[idx]).get_fdata()
    # temp_image_t2 = scaler.fit_transform(
    #     temp_image_t2.reshape(-1, temp_image_t2.shape[-1])
    # ).reshape(temp_image_t2.shape)
 
    # temp_image_flair = nib.load(flair_list[idx]).get_fdata()
    # temp_image_flair = scaler.fit_transform(
    #     temp_image_flair.reshape(-1, temp_image_flair.shape[-1])
    # ).reshape(temp_image_flair.shape)
 
    temp_mask = nib.load(mask_list[idx]).get_fdata()
 
    # temp_combined_images = np.stack(
    #     [temp_image_flair, temp_image_t1ce, temp_image_t2], axis=3
    # )
    temp_combined_images = np.expand_dims(temp_image_t1ce, 3)
 
    # temp_combined_images = temp_combined_images[56:184, 56:184, 13:141]
    # temp_mask = temp_mask[56:184, 56:184, 13:141]
 
    val, counts = np.unique(temp_mask, return_counts=True)
 
    # If a volume has less than 1% of mask, we simply ignore to reduce computation
    # if (1 - (counts[0] / counts.sum())) > 0.001:
    if True:
        #         print("Saving Processed Images and Masks")
        temp_mask = F.one_hot(torch.tensor(temp_mask, dtype=torch.long), num_classes=4)
        os.makedirs(f"{base_dir}/images", exist_ok=True)
        os.makedirs(f"{base_dir}/masks", exist_ok=True)
 
        np.save(
            f"{base_dir}/images/image_"
            + str(idx)
            + ".npy",
            temp_combined_images,
        )
        np.save(
            f"{base_dir}/masks/mask_"
            + str(idx)
            + ".npy",
            temp_mask,
        )
 
    else:
        pass

Preparing to stack, crop and save: 100%|██████████| 99/99 [4:34:35<00:00, 166.42s/file]     


In [3]:
images_folder = f"{base_dir}/images"
print(len(os.listdir(images_folder)))
 
masks_folder = f"{base_dir}/masks"
print(len(os.listdir(masks_folder)))

600
600


In [5]:
import splitfolders
input_folder = f"{base_dir}/"
 
output_folder = f"{base_dir}_1/"
 
splitfolders.ratio(
    input_folder, output_folder, seed=42, ratio=(0.75, 0.2, 0.05), group_prefix=None
)

Copying files: 965 files [01:00,  3.18 files/s]

OSError: [Errno 28] No space left on device: '/home/jincan/long_seg/my3DUNet/Data/Brats2024/single_t1c_noCrop/masks/mask_335.npy' -> '/home/jincan/long_seg/my3DUNet/Data/Brats2024/single_t1c_noCrop_1/train/masks/mask_335.npy'

In [3]:
import os
import shutil

base_dir = "/home/jincan/long_seg/my3DUNet/Data/Imaging_for_3DUNet_long/val"
output_dir = "/home/jincan/long_seg/my3DUNet/Data/Imaging_for_3DUNet_pair/val"

subj_dir = sorted(os.listdir(base_dir))
for subj in subj_dir:
    subj_path = os.path.join(base_dir, subj)
    tp_list = sorted(os.listdir(subj_path))
    for i in range(len(tp_list)-1):
        tp1 = tp_list[i]
        tp2 = tp_list[i+1]
        tp1_path = os.path.join(subj_path, tp1)
        tp2_path = os.path.join(subj_path, tp2)
        img1_path = os.path.join(tp1_path, "CT1_image_rig.nii.gz")
        mask1_path = os.path.join(tp1_path, "CT1_mask_rig.nii.gz")
        img2_path = os.path.join(tp2_path, "CT1_image_rig.nii.gz")
        mask2_path = os.path.join(tp2_path, "CT1_mask_rig.nii.gz")

        new_tp_folder_name = tp1 + "~"+ tp2[17:]
        print(new_tp_folder_name)
        new_img_folder = os.path.join(output_dir, "images", new_tp_folder_name)
        os.makedirs(new_img_folder, exist_ok=True)
        new_mask_folder = os.path.join(output_dir, "labels", new_tp_folder_name)
        os.makedirs(new_mask_folder, exist_ok=True)
        new_img1_file = os.path.join(new_img_folder, "image1_rig.nii.gz")
        new_img2_file = os.path.join(new_img_folder, "image2_rig.nii.gz")
        shutil.copy(img1_path, new_img1_file)
        shutil.copy(img2_path, new_img2_file)
        new_mask1_file = os.path.join(new_mask_folder, "mask1_rig.nii.gz")
        new_mask2_file = os.path.join(new_mask_folder, "mask2_rig.nii.gz")
        shutil.copy(mask1_path, new_mask1_file)
        shutil.copy(mask2_path, new_mask2_file)


Patient-077_week-000~001
Patient-077_week-001~014
Patient-077_week-014~026
Patient-077_week-026~033
Patient-077_week-033~048
Patient-077_week-048~062
Patient-077_week-062~070
Patient-077_week-070~083
Patient-082_week-000-1~000-2
Patient-082_week-000-2~003
Patient-086_week-000~173-1
Patient-086_week-173-1~173-2
Patient-086_week-173-2~181
Patient-091_week-014~026
